https://huggingface.co/mrm8488/t5-base-finetuned-summarize-news?text=Le+jaune+%C3%A9tait+caf%C3%A9t%C3%A9ria.+J%27ai+remis+les+lumi%C3%A8res.+J%27ai+vu+le+noir+vent.+Je+suis+all%C3%A9+admin.+J%27ai+crois%C3%A9+le+rouge+weapons.+J%27ai+crois%C3%A9+le+vert.+J%27ai+fait+les+fils.+J%27ai+tu%C3%A9+le+bleu.+Je+suis+all%C3%A9+navigation.+Je+suis+all%C3%A9+medbay.+J%27ai+tu%C3%A9+le+rouge.

https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip

     |████████████████████████████████| 1.6MB 8.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
!pip install transformers[sentencepiece]
!pip install wandb -q

     |████████████████████████████████| 2.2 MB 6.5 MB/s 
     |████████████████████████████████| 895 kB 54.0 MB/s 
     |████████████████████████████████| 3.3 MB 68.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.7 MB/s 
     |████████████████████████████████| 1.8 MB 8.4 MB/s 
     |████████████████████████████████| 131 kB 70.6 MB/s 
     |████████████████████████████████| 97 kB 7.7 MB/s 
     |████████████████████████████████| 166 kB 64.6 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [ ]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

In [ ]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Thu May 20 22:59:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
# Login to wandb to log the model run and all the parameters
!wandb login

wandb: Currently logged in as: denatflore (use `wandb login --relogin` to force relogin)


In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:

# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                # max_length=50,
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
def main():
    # WandB – Initialize a new run
    wandb.init(project="transformers_tutorials_summarization")

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    # config = wandb.config          # Initialize config
    # config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
    # config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
    # config.TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
    # config.VAL_EPOCHS = 1 
    # config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    # config.SEED = 42               # random seed (default: 42)
    # config.MAX_LEN = 512
    # config.SUMMARY_LEN = 150 

    # WandB – Config is a variable that holds and saves hyperparameters and inputs
    # Defining some key variables that will be used later on in the training  
    config = wandb.config          # Initialize config
    config.TRAIN_BATCH_SIZE = 5    # input batch size for training (default: 64)
    config.VALID_BATCH_SIZE = 5    # input batch size for testing (default: 1000)
    config.TRAIN_EPOCHS = 30        # number of epochs to train (default: 10)
    config.VAL_EPOCHS = 1 
    config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
    config.SEED = 42               # random seed (default: 42)
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150 

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(config.SEED) # pytorch random seed
    np.random.seed(config.SEED) # numpy random seed
    torch.backends.cudnn.deterministic = True

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    

    # Importing and Pre-Processing the domain data
    # Selecting the needed columns only. 
    # Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
    # df = pd.read_csv('/content/drive/Shareddrives/PFE_CHECK/Generation/news_summary.csv',encoding='latin-1')
    # df = pd.read_csv('/content/drive/Shareddrives/PFE_CHECK/Generation/news_summary_2.csv',encoding='latin-1')
    df = pd.read_csv('/content/drive/Shareddrives/PFE_CHECK/Generation/BDD_csv.csv',encoding='latin-1')
    df = df[['text','ctext']]
    df.ctext = 'summarize: ' + df.ctext
    print(df.head())

    
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
    train_size = 0.8
    train_dataset=df.sample(frac=train_size,random_state = config.SEED)
    val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print("FULL Dataset: {}".format(df.shape))
    print("TRAIN Dataset: {}".format(train_dataset.shape))
    print("TEST Dataset: {}".format(val_dataset.shape))


    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Defining the parameters for creation of dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)


    
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    # model = T5ForConditionalGeneration.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("/content/drive/Shareddrives/PFE_CHECK/Generation/test_model_T5_tuto")
    model = model.to(device)

    # Defining the optimizer that will be used to tune the weights of the network in the training session. 
    optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

    # Log metrics with wandb
    wandb.watch(model, log="all")
    # Training loop
    print('Initiating Fine-Tuning for the model on our dataset')

    for epoch in range(config.TRAIN_EPOCHS):
        train(epoch, tokenizer, model, device, training_loader, optimizer)


    # Validation loop and saving the resulting file with predictions and acutals in a dataframe.
    # Saving the dataframe as predictions.csv
    print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
    for epoch in range(config.VAL_EPOCHS):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
        final_df.to_csv('/content/drive/Shareddrives/PFE_CHECK/Generation/predictions_final_model.csv')
        print('Output Files generated for review')

    model.save_pretrained("/content/drive/Shareddrives/PFE_CHECK/Generation/T5_summarization_model")

if __name__ == '__main__':
    main()

wandb: Currently logged in as: denatflore (use `wandb login --relogin` to force relogin)



                                                text                                              ctext
0  I went storage, cafeteria and electrical. I cr...  summarize: I walked the floor of storage. I ha...
1  I started with my upper engine task, I crossed...  summarize: I entered upper engine. I finished ...
2  I did my cafeteria task, then I was with Orang...  summarize: I saw Lime, Green, Yellow, Blue and...
3  I walked around upper engine with Blue, Lime a...  summarize: I was walking around in the upper e...
4  The life of me I spent storage, admin and cafe...  summarize: The life of me was in storage. Oran...
FULL Dataset: (10, 2)
TRAIN Dataset: (8, 2)
TEST Dataset: (2, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  3.575972080230713
Epoch: 1, Loss:  2.8980917930603027
Epoch: 2, Loss:  2.386510133743286
Epoch: 3, Loss:  2.0290820598602295
Epoch: 4, Loss:  1.7565603256225586
Epoch: 5, Loss:  1.7417837381362915
Epoch: 6, Loss:  1.70562744140625
Epoch: 7, Loss:  1.2801822423934937
Epoch: 8, Loss:  1.0801310539245605
Epoch: 9, Loss:  1.2471195459365845
Epoch: 10, Loss:  1.1724088191986084
Epoch: 11, Loss:  1.0294770002365112
Epoch: 12, Loss:  0.7718218564987183
Epoch: 13, Loss:  0.8742697238922119
Epoch: 14, Loss:  0.6774073243141174
Epoch: 15, Loss:  0.5086314082145691
Epoch: 16, Loss:  0.5021819472312927
Epoch: 17, Loss:  0.46938905119895935
Epoch: 18, Loss:  0.45980796217918396
Epoch: 19, Loss:  0.3837950825691223
Epoch: 20, Loss:  0.34843307733535767
Epoch: 21, Loss:  0.33460021018981934
Epoch: 22, Loss:  0.25399285554885864
Epoch: 23, Loss:  0.24646000564098358
Epoch: 24, Loss:  0.17885980010032654
Epoch: 25, Loss:  0.19206884503364563
Epoch: 26, Loss:  0.2746170163154602
Epoch: 

# TESTING

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade pip

     |████████████████████████████████| 1.6MB 4.0MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
!pip install transformers[sentencepiece]
!pip install wandb -q

     |████████████████████████████████| 2.2 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 40.2 MB/s 
     |████████████████████████████████| 895 kB 40.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.5 MB/s 
     |████████████████████████████████| 1.8 MB 4.0 MB/s 
     |████████████████████████████████| 131 kB 40.1 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 166 kB 37.7 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 


In [ ]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# WandB – Import the wandb library
import wandb

In [ ]:
# Checking out the GPU we have access to. This is output is from the google colab version. 
!nvidia-smi

Thu May 20 15:14:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

model = T5ForConditionalGeneration.from_pretrained("/content/drive/Shareddrives/PFE_CHECK/Generation/T5_summarization_model")
model = model.to(device)

In [ ]:
# def summarize(text, max_length=50):
#   input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)

#   generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length,  repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)

#   preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

#   return preds[0]

In [ ]:
text = """I walked the floor of storage. I have walked the floor of cafeteria. I entered storage. I entered electrical. I entered storage. I saw Black. I saw more Black. I entered electrical. I have finished the Calibrate Distributor task. I entered storage. I entered shields. I entered oxygen. I entered weapons. I entered cafeteria. I entered admin. I saw Blue. I saw more Blue. I finished the Swipe Card task. I walked the floor of storage. I walked the floor of electrical. I have finished the Calibrate Distributor task. I entered storage. I have walked on shields soil. I entered oxygen. I have walked on shields soil. I walked the floor of storage. I walked the floor of electrical. I have finished the Calibrate Distributor task. I walked the floor of storage. I entered shields. I entered oxygen. I entered shields. I walked the floor of storage. I walked the floor of electrical. I have finished the Calibrate Distributor task. I finished the Divert Power task. I entered lower engine. I have stepped on the floor of security. I saw Green. I saw more Green. I finished the Accept Power (security) task.
"""

In [ ]:
def generate(tokenizer, model, device, text):
    MAX_LEN = 512
    SUMMARY_LEN = 50 
    VALID_BATCH_SIZE = 5

    val_dataset= pd.DataFrame([["",text]],columns=["text","ctext"])

    val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

    val_params = {
            'batch_size': VALID_BATCH_SIZE,
            'shuffle': False,
            'num_workers': 0
            }

    loader = DataLoader(val_set, **val_params)
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]

            predictions.extend(preds)
    return predictions

In [ ]:
generate(tokenizer, model, device, text)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


'storage, cafeteria and electrical. I crossed Black in storage and Blue in admin while going to do Swipe Card task. At the end I was doing my tasks at lower engine then electrical and I crossed paths with Green. Arnaud is suspect.'

In [ ]:
text = 'storage, cafeteria and electrical. I crossed Black in storage and Blue in admin while going to do Swipe Card task. At the end I was doing my tasks at lower engine then electrical and I crossed paths with Green. Arnaud is suspect. test pb fin'

In [ ]:
text.split(".")

['storage, cafeteria and electrical',
 ' I crossed Black in storage and Blue in admin while going to do Swipe Card task',
 ' At the end I was doing my tasks at lower engine then electrical and I crossed paths with Green',
 ' Arnaud is suspect',
 ' test pb fin']

In [ ]:
if text[0].isupper():
  newText = ".".join(text.split(".")[:-1])
else:
  newText = ".".join(text.split(".")[1:-1])
newText

' I crossed Black in storage and Blue in admin while going to do Swipe Card task. At the end I was doing my tasks at lower engine then electrical and I crossed paths with Green. Arnaud is suspect'

# Score

In [1]:
#rouge scores for a reference/generated sentence pair
#source google seq2seq source code.

import itertools

#supporting function
def _split_into_words(sentences):
  """Splits multiple sentences into words and flattens the result"""
  return list(itertools.chain(*[_.split(" ") for _ in sentences]))

#supporting function
def _get_word_ngrams(n, sentences):
  """Calculates word n-grams for multiple sentences.
  """
  assert len(sentences) > 0
  assert n > 0

  words = _split_into_words(sentences)
  return _get_ngrams(n, words)

#supporting function
def _get_ngrams(n, text):
  """Calcualtes n-grams.
  Args:
    n: which n-grams to calculate
    text: An array of tokens
  Returns:
    A set of n-grams
  """
  ngram_set = set()
  text_length = len(text)
  max_index_ngram_start = text_length - n
  for i in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[i:i + n]))
  return ngram_set

def rouge_n(reference_sentences, evaluated_sentences, n=2):
  """
  Computes ROUGE-N of two text collections of sentences.
  Source: http://research.microsoft.com/en-us/um/people/cyl/download/
  papers/rouge-working-note-v1.3.1.pdf
  Args:
    evaluated_sentences: The sentences that have been picked by the summarizer
    reference_sentences: The sentences from the referene set
    n: Size of ngram.  Defaults to 2.
  Returns:
    recall rouge score(float)
  Raises:
    ValueError: raises exception if a param has len <= 0
  """
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("Collections must contain at least 1 sentence.")

  evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
  reference_ngrams = _get_word_ngrams(n, reference_sentences)
  reference_count = len(reference_ngrams)
  evaluated_count = len(evaluated_ngrams)

  # Gets the overlapping ngrams between evaluated and reference
  overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
  overlapping_count = len(overlapping_ngrams)

  # Handle edge case. This isn't mathematically correct, but it's good enough
  if evaluated_count == 0:
    precision = 0.0
  else:
    precision = overlapping_count / evaluated_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlapping_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  #just returning recall count in rouge, useful for our purpose
  return recall,precision,f1_score

In [2]:
# Text and generation from validation space.
realText = """I walked around upper engine with Blue, Lime and Orange, then I did my cafeteria and storage tasks. I crossed Orange to electrical, then Green to reactor. When I returned to electrical I saw Lime dead, it's Orange for sure. Arnaud is suspect."""
generated = """I crossed Lime's backbone while walking around the upper engine, my heart open to the unknown. At the reactor, I ran into Vert and I crossed Lime's backbone. Then I crossed Lime's backbone. Arnaud is suspect."""
realText_2 = """The life of me I was at Admin, Cafeteria and O2 with my heart open to the unknown. Then I walked on the floor of Security, Lower Engine and Admin. Finaly I walked around Cafeteria, Weapons, O2 and Shields with my heart open to the unknown. Arnaud is suspect."""
generated_2 = """I finished my tasks Clear Asteroids, Shields and Prime Shields. The life of me I was then at Admin and Cafeteria with Blue. Then I crossed Blue at None. I finished my tasks. I entered Medbay, Upper Engine, Reactor, Lower Engine, Electrical and I was at Storage. The life of me I was then at Navigation. I entered Shields, Communications and I entered Storage. The life of me I was then at Navigation. I entered Weapons and I entered O2. The life of me"""

In [12]:
recall,precision,f1_score = rouge_n(realText, generated, n=2)
print("Rouge-2 recall : " + str(recall))
print("Rouge-2 precision : " + str(precision))
print("Rouge-2 f1_score : " + str(f1_score))

Rouge-2 recall : 0.5952380952380952
Rouge-2 precision : 0.7142857142857143
Rouge-2 f1_score : 0.6493506443919718


In [13]:
recall,precision,f1_score = rouge_n(realText_2, generated_2, n=2)
print("Rouge-2 recall : " + str(recall))
print("Rouge-2 precision : " + str(precision))
print("Rouge-2 f1_score : " + str(f1_score))

Rouge-2 recall : 0.71875
Rouge-2 precision : 0.5859872611464968
Rouge-2 f1_score : 0.6456140301394891
